In [21]:
import quiverquant
import pandas as pd

#My Quiver API token. Replace this with a string of your own.
token = load_obj("APIToken")

quiver = quiverquant.quiver(token)
df = quiver.wallstreetbetsComments(date_from="20180901")
df

Pulling data from:  https://api.quiverquant.com/beta/live/wsbcomments?date_from=20180901


,Ticker,Puts,Calls,Time,Sentiment,Datetime
0,AAPL,0,0,1572592759000,0.0000,2019-11-01 07:19:19
1,ATVI,3,0,1572592825000,-0.2230,2019-11-01 07:20:25
2,BABA,0,0,1572592866000,0.5719,2019-11-01 07:21:06
3,GLD,0,0,1572593101000,0.3182,2019-11-01 07:25:01
4,PINS,0,0,1572593120000,0.5562,2019-11-01 07:25:20
...,...,...,...,...,...,...
3319415,PINS,0,0,1572592290000,0.2263,2019-11-01 07:11:30
3319416,BABA,1,2,1572592354000,0.5994,2019-11-01 07:12:34
3319417,AMD,0,0,1572592510000,0.7184,2019-11-01 07:15:10
3319418,SPY,0,0,1572592602000,0.0000,2019-11-01 07:16:42


In [3]:
df["Date"] = df["Datetime"].dt.date
dfDay = df.groupby(["Ticker","Date"]).count().reset_index()
dfDay = dfDay.rename(columns={"Puts": "Mentions"})
dfDay = dfDay[["Ticker", "Date", "Mentions"]]
dfDay

,Ticker,Date,Mentions
0,AA,2018-09-07,1
1,AA,2018-10-09,1
2,AA,2018-10-18,1
3,AA,2018-11-28,1
4,AA,2019-01-04,1
...,...,...,...
267796,ZYXI,2019-08-01,1
267797,ZYXI,2020-01-09,1
267798,ZYXI,2020-02-24,1
267799,ZYXI,2020-05-06,1


In [4]:
dfDay["Date"] = pd.to_datetime(dfDay["Date"])
dfWeek = dfDay.groupby([pd.Grouper(key='Date', freq='W-MON'), 'Ticker'])['Mentions'].sum().reset_index().sort_values('Date')
dfWeek

,Date,Ticker,Mentions
0,2018-09-03,AAPL,3
22,2018-09-03,LEN,2
23,2018-09-03,LOW,1
24,2018-09-03,MDB,1
25,2018-09-03,MSFT,2
...,...,...,...
102910,2021-02-22,FREE,11
102909,2021-02-22,FR,1
102908,2021-02-22,FOX,3
102916,2021-02-22,FSK,1


In [5]:
import yfinance as yf
import datetime as dt
dfLarge = dfWeek[dfWeek["Mentions"]>1]

dfLarge = dfLarge.sort_values("Date", ascending=True)
dates = dfLarge["Date"].unique()


capital = 100000
started = False
startedDFW = False

for date in dates[:-2]:
    dfW = dfLarge[dfLarge["Date"]==date]
    dfW = dfW.sort_values("Mentions", ascending=False).head(5)
    dfW['prop'] = dfW['Mentions']/dfW["Mentions"].sum()
    dfW['buy'] = capital*dfW['prop']
    buyDate = date+pd.Timedelta(days=6)
    dfW['buyDate'] = [buyDate]*len(dfW['buy'])
    if not startedDFW:
        dfWs = dfW
        startedDFW = True
    else:
        dfWs = pd.concat([dfWs, dfW])
    sellDate = date+pd.Timedelta(days=15)

    startedWeek = False
    print(date)
    for index, row in dfW.iterrows():
        ticker = row["Ticker"]
        print(ticker)
        try:
            ytStock = yf.download(ticker, start=str(buyDate.date()), end=str(sellDate.date()), interval="1d").reset_index()
            shares = row["buy"]/ytStock["Adj Close"].values[0]
            ytStock = ytStock.iloc[1:]

        except:
            print("Error")
            ytStock = yf.download("SPY", start=str(buyDate.date()), end=str(sellDate.date()), interval="1d").reset_index()
            shares = row["buy"]/ytStock["Adj Close"].values[0]
            ytStock = ytStock.iloc[1:]

        ytStock["OpenAmount"] = ytStock["Open"]*shares
        ytStock["CloseAmount"] = ytStock["Adj Close"]*shares
        ytStock["Ticker"] = [ticker]*len(ytStock["OpenAmount"])
        ytStock = ytStock.fillna(method='ffill')
        ytStock = ytStock.fillna(method='bfill')
        ytStock = ytStock.dropna()
        if not startedWeek:
            dfCombined = ytStock
            startedWeek = True
        else:
            dfCombined = pd.concat([dfCombined, ytStock])
        
        if not started:
            dfAll = ytStock
            started = True
        else:
            dfAll = pd.concat([dfAll, ytStock])
            
    capital = 0
    for ticker in dfCombined["Ticker"].unique():
        dfT = dfCombined[dfCombined["Ticker"]==ticker]
        capital+=dfT["CloseAmount"].values[-1]
    print("Week end capital: ", capital)



2018-09-03T00:00:00.000000000
MU
[*********************100%***********************]  1 of 1 completed
EA
[*********************100%***********************]  1 of 1 completed
TSLA
[*********************100%***********************]  1 of 1 completed
USO
[*********************100%***********************]  1 of 1 completed
AMZN
[*********************100%***********************]  1 of 1 completed
Week end capital:  99874.49121246433
2018-09-10T00:00:00.000000000
AMD
[*********************100%***********************]  1 of 1 completed
MU
[*********************100%***********************]  1 of 1 completed
MSFT
[*********************100%***********************]  1 of 1 completed
AMZN
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
Week end capital:  101603.08988989101
2018-09-17T00:00:00.000000000
AMD
[*********************100%***********************]  1 of 1 completed
MU
[*********************100%*

[*********************100%***********************]  1 of 1 completed
Week end capital:  76704.98047281272
2019-01-14T00:00:00.000000000
AMD
[*********************100%***********************]  1 of 1 completed
SPY
[*********************100%***********************]  1 of 1 completed
TLRY
[*********************100%***********************]  1 of 1 completed
NFLX
[*********************100%***********************]  1 of 1 completed
BBBY
[*********************100%***********************]  1 of 1 completed
Week end capital:  78878.55812046285
2019-01-21T00:00:00.000000000
SPY
[*********************100%***********************]  1 of 1 completed
NFLX
[*********************100%***********************]  1 of 1 completed
MS
[*********************100%***********************]  1 of 1 completed
BAC
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
Week end capital:  82349.20386202379
2019-01-28T00:00:00.0000000

[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
Week end capital:  83944.87920555833
2019-05-27T00:00:00.000000000
SPY
[*********************100%***********************]  1 of 1 completed
TSLA
[*********************100%***********************]  1 of 1 completed
ROKU
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
EOD
[*********************100%***********************]  1 of 1 completed
Week end capital:  94196.86236690548
2019-06-03T00:00:00.000000000
SPY
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
BYND
[*********************100%***********************]  1 of 1 completed
TSLA
[*********************100%***********************]  1 of 1 completed
EOD
[*********************100%***********************]  1 of 1

[*********************100%***********************]  1 of 1 completed
DIS
[*********************100%***********************]  1 of 1 completed
ROKU
[*********************100%***********************]  1 of 1 completed
Week end capital:  89997.97061606767
2019-10-07T00:00:00.000000000
SPY
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
MSFT
[*********************100%***********************]  1 of 1 completed
COST
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
Week end capital:  91503.65954783607
2019-10-14T00:00:00.000000000
SPY
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
EOD
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1

[*********************100%***********************]  1 of 1 completed
SPY
[*********************100%***********************]  1 of 1 completed
DIS
[*********************100%***********************]  1 of 1 completed
SPCE
[*********************100%***********************]  1 of 1 completed
Week end capital:  125644.47382638174
2020-02-17T00:00:00.000000000
MSFT
[*********************100%***********************]  1 of 1 completed
SPCE
[*********************100%***********************]  1 of 1 completed
TSLA
[*********************100%***********************]  1 of 1 completed
SPY
[*********************100%***********************]  1 of 1 completed
BABA
[*********************100%***********************]  1 of 1 completed
Week end capital:  114968.1642666916
2020-02-24T00:00:00.000000000
SPCE
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
MSFT
[*********************100%***********************]  1 o

[*********************100%***********************]  1 of 1 completed
BA
[*********************100%***********************]  1 of 1 completed
MSFT
[*********************100%***********************]  1 of 1 completed
KR
[*********************100%***********************]  1 of 1 completed
EOD
[*********************100%***********************]  1 of 1 completed
Week end capital:  103196.81952192134
2020-06-29T00:00:00.000000000
SPY
[*********************100%***********************]  1 of 1 completed
FB
[*********************100%***********************]  1 of 1 completed
BA
[*********************100%***********************]  1 of 1 completed
EOD
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
Week end capital:  101682.45605207543
2020-07-06T00:00:00.000000000
TSLA
[*********************100%***********************]  1 of 1 completed
SPY
[*********************100%***********************]  1 of 1 comp

[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
NIO
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
MSFT
[*********************100%***********************]  1 of 1 completed
Week end capital:  125226.81915631304
2020-11-09T00:00:00.000000000
NIO
[*********************100%***********************]  1 of 1 completed
SPY
[*********************100%***********************]  1 of 1 completed
ACB
[*********************100%***********************]  1 of 1 completed
BABA
[*********************100%***********************]  1 of 1 completed
RKT
[*********************100%***********************]  1 of 1 completed
Week end capital:  135964.04770996037
2020-11-16T00:00:00.000000000
NIO
[*********************100%***********************]  1 of 1 completed
PLTR
[*********************100%***********************]  1 of 

In [6]:
dfDay = dfAll.groupby("Date").sum().reset_index()
dfDay = dfDay[dfDay["Date"]!=pd.to_datetime("2020-8-31")]
dfDay["Fund"] = ["WSB"]*len(dfDay["Close"])
dfDay

,Date,Open,High,Low,Close,Adj Close,Volume,OpenAmount,CloseAmount,Fund
0,2018-09-11,2254.264027,2319.250008,2238.429993,2315.728027,2315.578598,105633000,98602.876081,99282.779572,WSB
1,2018-09-12,2321.538002,2334.080002,2288.359936,2321.088001,2320.937626,126806363,98141.567929,99321.659858,WSB
2,2018-09-13,2330.533997,2344.220013,2309.516029,2321.141987,2320.990498,100055038,99531.099307,100259.117586,WSB
3,2018-09-14,2325.042053,2331.866028,2288.213974,2303.799938,2303.647877,83332550,100660.744792,101092.077546,WSB
4,2018-09-17,2287.587975,2293.333942,2216.266037,2239.028034,2238.878071,75012413,100323.973803,99874.491212,WSB
...,...,...,...,...,...,...,...,...,...,...
599,2021-02-10,172.580000,185.980000,167.669999,173.120000,173.120000,295784000,291311.772160,293176.574373,WSB
600,2021-02-11,171.500001,177.300003,168.760004,172.140001,172.140001,154264600,287004.383414,286948.420054,WSB
601,2021-02-12,171.719998,177.570000,168.609998,174.050000,174.050000,128979800,287221.285067,292418.399412,WSB
602,2021-02-17,99.240000,102.479998,91.940002,93.999999,93.999999,273156400,291268.591431,277727.075341,WSB


In [20]:
dfSPY = yf.download("SPY", start="2018-9-12", end="2021-02-18", interval="1d").reset_index()
dfSPY["Fund"] = ["S&P 500"] * len(dfSPY["Open"])
shares = 100000/dfSPY["Open"].values[0]
dfSPY["OpenAmount"] = dfSPY["Open"]*shares
dfSPY["CloseAmount"] = dfSPY["Close"]*shares
dfCombined = pd.concat([dfDay, dfSPY])
dfCombined

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume,OpenAmount,CloseAmount,Fund
0,2018-09-11,2254.264027,2319.250008,2238.429993,2315.728027,2315.578598,105633000,98602.876081,99282.779572,WSB
1,2018-09-12,2321.538002,2334.080002,2288.359936,2321.088001,2320.937626,126806363,98141.567929,99321.659858,WSB
2,2018-09-13,2330.533997,2344.220013,2309.516029,2321.141987,2320.990498,100055038,99531.099307,100259.117586,WSB
3,2018-09-14,2325.042053,2331.866028,2288.213974,2303.799938,2303.647877,83332550,100660.744792,101092.077546,WSB
4,2018-09-17,2287.587975,2293.333942,2216.266037,2239.028034,2238.878071,75012413,100323.973803,99874.491212,WSB
...,...,...,...,...,...,...,...,...,...,...
607,2021-02-10,392.119995,392.279999,387.500000,390.079987,390.079987,59154400,135653.497000,134947.758205,S&P 500
608,2021-02-11,391.239990,391.690002,388.100006,390.709991,390.709991,42755400,135349.060243,135165.707727,S&P 500
609,2021-02-12,389.850006,392.899994,389.769989,392.640015,392.640015,50505700,134868.196705,135833.397206,S&P 500
610,2021-02-16,393.959991,394.170013,391.529999,392.299988,392.299988,50700800,136290.041785,135715.765276,S&P 500


In [19]:
import plotly.express as px
import plotly 
#dfCombined = dfCombined[dfCombined["Date"]>"2021-01-28"]
fig = px.line(dfCombined, x="Date", y="CloseAmount", title='WSB', color="Fund", 
             color_discrete_sequence=["rgb(229, 81, 39)","rgb(118, 213, 232)" ])

wsbReturn = (capital-100000)/100000*100


fig.update_layout(title="<b>+"+str(round(wsbReturn, 2))+"% Return</b><br>Aug 2018 - Feb 2021", titlefont=dict(color='rgb(229, 81, 39)', size=20), plot_bgcolor='rgb(32,36,44)', paper_bgcolor='rgb(32,36,44)')
fig.update_xaxes(title_text="",color='white', showgrid=False, tickfont=dict(size=10))
fig.update_yaxes(title_text="$", color='white', showgrid=False, titlefont=dict(size=20),gridcolor="rgb(228,49,34)")
fig.update_layout(
    legend=dict(
        title=dict(text="",font=dict(color='white')),
        x=.85, y=1.15,
        font=dict(
            color='white',
            size=15
        )
    )
)
fig.update_traces(line=dict(width=3))
for i in range(0, len(fig.data)):
    fig.data[i].hovertemplate = "<b>%{x}</b><br>$%{y:.0f}<extra></extra>"


fig.show()

In [17]:
import plotly.graph_objects as go


fig = go.Figure(px.bar(dfWs, x="buyDate", y="buy", color='Ticker',text='Ticker',color_discrete_sequence=px.colors.qualitative.Light24))

fig.update_layout(title="Portfolio by Week", titlefont=dict(color='rgb(235,60,92)'), plot_bgcolor='rgb(32,36,44)', paper_bgcolor='rgb(32,36,44)')
fig.update_xaxes(title_text="",color='white', showgrid=False, fixedrange=False)
fig.update_yaxes(title_text="$",color='white', showgrid=False,  fixedrange=False,gridwidth=1,gridcolor="rgb(109,177,174)")
fig.update_layout(
    legend=dict(
        title=dict(text="Ticker",font=dict(color="white")),
        
        font=dict(
            color='white'
        ),
        
    )
)


fig.show()

In [16]:
import plotly

dfTop = dfWs.groupby("Ticker").count().reset_index().sort_values("Mentions", ascending=False).head(15)
fig = go.Figure(px.bar(dfTop, x="Ticker", y="Mentions", text='Mentions',color_discrete_sequence=['rgb(118, 213, 232)']))

fig.update_layout(title="15 Most Frequently Held Stocks", titlefont=dict(color='rgb(228,49,34)'), plot_bgcolor='rgb(32,36,44)', paper_bgcolor='rgb(32,36,44)')
fig.update_xaxes(title_text="", fixedrange=True,color='white', showgrid=False)
fig.update_yaxes(title_text="Weeks Held",color='white', fixedrange=True, showgrid=False, gridwidth=1,gridcolor="rgb(109,177,174)")
fig.update_layout(
    legend=dict(
        title=dict(text="Ticker",font=dict(color="white")),
        
        font=dict(
            color='white'
        ),
        
    )
)
for i in range(0, len(fig.data)):
    fig.data[i].hovertemplate = "<b>%{x}</b><br>Count: %{text}<br><extra></extra>"

fig.show()
